# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#512 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846824086674                   -0.70    4.0
  2   -7.852304774566       -2.26       -1.53    1.0
  3   -7.852646083653       -3.47       -2.52    3.2
  4   -7.852646677099       -6.23       -3.34    2.2
  5   -7.852646685880       -8.06       -4.80    1.0
  6   -7.852646686726       -9.07       -5.18    4.2
  7   -7.852646686730      -11.46       -5.88    2.0
  8   -7.852646686730      -12.46       -7.34    2.0
  9   -7.852646686730   +    -Inf       -7.64    2.2
 10   -7.852646686730      -14.75       -8.90    1.2
 11   -7.852646686730   +  -14.45       -9.66    3.2
 12   -7.852646686730      -14.45      -10.05    1.0
 13   -7.852646686730   +  -15.05       -9.36    1.0
 14   -7.852646686730   +    -Inf       -8.85    1.0
 15   -7.852646686730   +  -15.05       -8.85    1.0
 16   -7.852646686730   +    -Inf       -9.03    1.0
 17   -7.852646686730   +  -14.75       -8.96 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846824170117                   -0.70           4.0
  2   -7.852523252023       -2.24       -1.62   0.80    4.8
  3   -7.852609674015       -4.06       -2.78   0.80    5.2
  4   -7.852646571306       -4.43       -3.56   0.80    7.2
  5   -7.852646683718       -6.95       -4.57   0.80    9.0
  6   -7.852646686684       -8.53       -5.14   0.80   10.8
  7   -7.852646686726      -10.38       -6.22   0.80   11.5
  8   -7.852646686730      -11.37       -7.11   0.80   14.0
  9   -7.852646686730      -13.55       -7.44   0.80   15.5
 10   -7.852646686730   +    -Inf       -8.13   0.80   15.2
 11   -7.852646686730   +  -14.75       -9.35   0.80   16.8
 12   -7.852646686730      -14.57      -10.25   0.80   18.5
 13   -7.852646686730   +  -14.57      -11.46   0.80   21.5
 14   -7.852646686730   +  -14.75      -11.42   0.80   23.2
 15   -7.852646686730   +    -Inf      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.372583e+01     3.374448e+00
 * time: 0.4179508686065674
     1     9.813259e-01     2.089029e+00
 * time: 0.6095130443572998
     2    -1.649382e+00     2.337571e+00
 * time: 0.6332089900970459
     3    -3.532183e+00     2.178909e+00
 * time: 0.6668858528137207
     4    -4.744484e+00     1.917159e+00
 * time: 0.7010250091552734
     5    -6.536682e+00     1.509836e+00
 * time: 0.7346508502960205
     6    -7.390211e+00     7.322079e-01
 * time: 0.7682590484619141
     7    -7.663449e+00     1.984115e-01
 * time: 0.7916078567504883
     8    -7.747018e+00     1.797654e-01
 * time: 0.8154230117797852
     9    -7.790499e+00     1.088309e-01
 * time: 0.8387110233306885
    10    -7.820574e+00     8.215206e-02
 * time: 0.8620619773864746
    11    -7.838691e+00     6.925646e-02
 * time: 0.8854939937591553
    12    -7.845003e+00     5.514011e-02
 * time: 0.9092559814453125
    13    -7.847868e+00     4.356015e-02
 * time: 0.9328520298

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846756738797                   -0.70    3.8
  2   -7.852282136320       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646685553                   -2.55
  2   -7.852646686730       -8.93       -5.42
  3   -7.852646686730   +  -14.57      -10.08
  4   -7.852646686730      -15.05      -15.72
|ρ_newton - ρ_scf|  = 1.4261848795118743e-13
|ρ_newton - ρ_scfv| = 2.312673963036395e-13
|ρ_newton - ρ_dm|   = 1.0599735325091614e-9
